# 

# NLTK

## Tokenize

In [1]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(some_text_data)

## Stop Words

In [ ]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english')

stopwords_list = stopwords.words('english')

stopwords_list += list(string.punctuation)

stopwords_list += [str(i) for i in range(0,10)]

macbeth_words_stopped = [word for word in macbeth_tokens if word not in stopwords_list]

## Frequency Distributions

In [ ]:
nltk.FreqDist()

macbeth_stopped_freqdist = FreqDist(macbeth_words_stopped)

macbeth_stopped_freqdist.most_common(50)

# TF-IDF Vectorization

- If the word is used often in many other documents, it is not unique, and therefore probably not too useful if we wanted to figure out how this document is unique in relation to other documents. 

- Conversely, if a word is used many times in a document, but rarely in all the other documents we are considering, then it is likely a good indicator for telling us that this word is important to the document in question.  

The formula TF-IDF uses to determine the weights of each term in a document is **_Term Frequency_** multiplied by **_Inverse Document Frequency_**, where the formula for Term Frequency is:

$$\large Term\ Frequency(t) = \frac{number\ of\ times\ t\ appears\ in\ a\ document} {total\ number\ of\ terms\ in\ the\ document} $$

The formula for Inverse Document Frequency is:  
<br>  
<br>
$$\large  IDF(t) =  log_e(\frac{Total\ Number\ of\ Documents}{Number\ of\ Documents\ with\ t\ in\ it})$$



# t-SNE

In [ ]:
t_sne_object_3d = TSNE(n_components=3)
transformed_data_3d = t_sne_object_3d.fit_transform(tf_idf_vals_list)

# Regular Expressions

<img src="./images/regex_cheat_sheet.png">

In [ ]:
import re

pattern = '\d'
p = re.compile(pattern)
digits = p.findall(file)

Since the dollar sign is a reserved symbol in regex, we'll need to escape it using a \. This tells the regex compiler that we are talking about the actual dollar sign symbol, not using it to talk about the end of a string.